In [2]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import beam_search

In [3]:
# juana la loca

# with open("20099-8.txt", encoding = "latin-1") as file:
#     text = file.read()
# print(len(text))
# print(text[:1000])

In [4]:
# with open("64763-0.txt", encoding = "utf-8") as file:
#     text = file.read()
# print(len(text))
# print(text[:1000])

In [5]:
# chess

with open("data/pg5614.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:100000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [6]:
# with open("seq2seq.py", encoding = "utf-8") as file:
#     text = file.read()
# print(len(text))
# print(text[:1000])

In [7]:
len(text)

556949

In [8]:
vocabulary = set(text)
len(vocabulary)

92

In [9]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
print(len(char2i))
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
print(len(i2char))

95
95


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [11]:
length = 20
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)
source_1 = encoded[:, :length // 2]
print(source_1.shape)
target_1 = torch.cat((torch.ones(encoded.shape[0], 1).to(device).long(), encoded[:, length // 2:]), axis = 1)
print(target_1.shape)

556949
556929
['\ufeffThe Project Gutenbe', 'The Project Gutenber', 'he Project Gutenberg', 'e Project Gutenberg ', ' Project Gutenberg E']


  0%|          | 0/556929 [00:00<?, ?it/s]

torch.Size([556929, 20])
torch.Size([556929, 10])
torch.Size([556929, 11])


In [16]:
importlib.reload(beam_search)    
net = beam_search.Seq2SeqRNN(char2i, i2char, encoder_hidden_units = 100, decoder_hidden_units = 100)
net.to(device)
# net.load_state_dict(torch.load("models/save_transformer.pt"))
# net.eval()
# pprint([net.tensor2text(t) for t in net.beam_search(source_1[:2], 
#                                                     max_predictions = 5,
#                                                     candidates = 3,
#                                                     beam_width = 4)[0][:, :, 1:]])
# net.fit(source_1[:10], target_1[:10], epochs = 50, batch_size = 150, lr = 0.0001, verbose = 3)
net.fit(source_1, target_1, epochs = 50, batch_size = 150, lr = 0.0001, verbose = 3)
# net.fit(source_2, target_2, epochs = 10, batch_size = 100, lr = 0.00001, verbose = 2)
# net.fit(source_3, target_3, epochs = 10, batch_size = 100, lr = 0.00001)

Net parameters: 207,195


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/3713 [00:00<?, ?it/s]

Epoch:    1, Loss: 2.5302
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' eh  t   e', ' h  te  e ', '   eh  e e', '  th  e e ', ' th  e e  ']
greedy_search
['          ', '          ', '          ', '          ', '          ']
sample
['no\nice amU', 'gcithsspt ', 'ae the usw', 'h in gpitf', 'k a/  he  ']
beam_search
[['          ', '          ', '          ', '          ', '          '],
 ['         t', '         t', '        e ', 'e         ', 'e         '],
 ['        e ', '         a', '         a', '         t', '         t'],
 ['         e', '         h', '         e', ' e        ', '         K'],
 ['         a', '         e', '         t', '        e ', '         .']]
tensor([-11.1200, -10.9700, -12.1900,  -9.9900,  -9.2400], device='cuda:0')
tensor([[-11.1200, -10.9700, -12.1900,  -9.9900,  -9.2400],
        [-13.9100, -13.9300, -14.3600, -12.0700, -11.4700],
        [-14.0500, -14.0700, -14.5200, -12.3900, -11.5400],
        [-14.1400, -14.09

  0%|          | 0/3713 [00:00<?, ?it/s]

...

Epoch:   25, Loss: 1.0084
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-L']
greedy_search
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-t']
sample
['ck dLay Cu', 't Gutenber', ' defencedo', '|\n   White', 'tefergremp']
beam_search
[['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-t'],
 ['t Gutenber', 't\nGutenber', ' Gutenber-', 'Gutenberg ', 'utenberg L'],
 ['ct Gutente', 't Gutenbee', ' Gutenber ', 'Gutenberg,', 'utenberg t'],
 ['ck Gutenbe', 't.\n\n      ', ' Gutenberd', "Gutenberg'", 'utenberg\nt'],
 ['ct Gutenge', 't Gutenbel', ' Gutenbele', 'Gutenberg.', 'utenberg.\n']]
tensor([-2.6700, -1.6100, -1.9400, -3.0100, -2.7000], device='cuda:0')
tensor([[-2.6700, -1.6100, -1.9400, -3.0100, -2.7000],
        [-4.0600, -3.9000, -5.6500, -3.7500, -4.7400],
        [-4.7200, -4.5600, -5.7900, -4.6800, -5.3700],
        [-5.0400, -4.8400, -5.8100, -4.7

  0%|          | 0/3713 [00:00<?, ?it/s]

Epoch:   26, Loss: 1.0013
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-L']
greedy_search
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-t']
sample
['ct Gutenbe', 't Gutenber', ' Gitingr, ', 'verioged.\n', ' 5. 6.\n\nTr']
beam_search
[['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-t'],
 ['t Gutenber', 't\nGutenber', ' Gutenber-', 'Gutenberg ', 'utenberg L'],
 ['ct Gutente', 't Gutenbee', ' Gutenber ', "Gutenberg'", 'utenberg-L'],
 ['ck Gutenbe', 't.\n\n      ', ' Gutenberd', 'Gutenberg.', 'utenberg.\n'],
 ['ct Gutenge', 't Gutenbel', ' Gutenbele', 'Gutenberg,', 'utenberg t']]
tensor([-2.5500, -1.4300, -1.6600, -2.7000, -2.3900], device='cuda:0')
tensor([[-2.5500, -1.4300, -1.6600, -2.7000, -2.3900],
        [-3.9700, -3.7000, -5.2700, -3.7300, -4.6100],
        [-4.7600, -4.5800, -5.5400, -4.4000, -5.2800],
        [-5.0000, -4.9000, -5.7000, -4.

  0%|          | 0/3713 [00:00<?, ?it/s]

KeyboardInterrupt: 